In [1]:
import requests
import pandas as pd
import numpy as np

In [2]:
def get_league_info(current_id):
    """Returns the league information from the sleeper api.

    Args:
        current_id (string or integer): A string/integer containing the most recent league id.

    Returns:
        dict: The league's current information.
    """

    league_url = f"https://api.sleeper.app/v1/league/{current_id}"
    response = requests.get(league_url)
    if response.status_code == 200:
        data = response.json()
    else:
        print(f"Error: {response.status_code}")

    return(data)

def get_all_league_info(league_id):
    """Generates a list of all league information from the sleeper api, calls the get_league_id function.

    Args:
        league_id (string or integer): A string/integer containing the most recent league id.

    Returns:
        dict: A dictionary containing all previous league info: id, season, playoff_week_start
    """

    while league_id != None:
        league_data = get_league_info(league_id)
        try:
            league_info["league_id"].append(league_id)
            league_info["season"].append(league_data["season"])
            league_info["playoff_week_start"].append(league_data["settings"]["playoff_week_start"])

            league_id = league_data["previous_league_id"]

        except NameError:
            # create league dictionary if it doesn't exist
            league_info = {"league_id": [],
                           "season": [],
                           "playoff_week_start":[]}
            
            

            league_info["league_id"].append(league_id)
            league_info["season"].append(league_data["season"])
            league_info["playoff_week_start"].append(league_data["settings"]["playoff_week_start"])

            league_id = league_data["previous_league_id"]


    
    return(pd.DataFrame(league_info))

In [3]:
def get_roster_id(league_id, year):
    users_dict = {}
    rosters_dict = {}

    users_url = f"https://api.sleeper.app/v1/league/{league_id}/users"
    rosters_url = f"https://api.sleeper.app/v1/league/{league_id}/rosters"

    # Make the API request
    users_response = requests.get(users_url)
    # Check if the request was successful (status code 200)
    if users_response.status_code == 200:
        # Parse the JSON response
        users = users_response.json()
    else:
        print(f"Error: {users_response.status_code}")

    # Make the API request
    rosters_response = requests.get(rosters_url)
    # Check if the request was successful (status code 200)
    if rosters_response.status_code == 200:
        # Parse the JSON response
        rosters = rosters_response.json()
    else:
        print(f"Error: {rosters_response.status_code}")

    for i, (user, roster) in enumerate(zip(users, rosters)):
        try:
            image_link = user["metadata"]["avatar"]
        except KeyError:
            image_link = "Not found."

        users_dict[i+1] = [user["display_name"], user["user_id"], image_link, year]
        rosters_dict[i+1] = [roster['owner_id'], roster['roster_id']]

    users_df = pd.DataFrame.from_dict(users_dict, orient = 'index', columns=['display_name', 'user_id', "image_link","Season"])
    rosters_df = pd.DataFrame.from_dict(rosters_dict, orient = 'index', columns=['user_id', 'roster_id'])

    merged_df = pd.merge(users_df, rosters_df, on = 'user_id')

    return(merged_df)

In [4]:
def get_matchups(league_id, week_number):
    api_url = f"https://api.sleeper.app/v1/league/{league_id}/matchups/{week_number}"
    # Make the API request
    response = requests.get(api_url)

    # Check if the request was successful (status code 200)
    if response.status_code == 200:
        # Parse the JSON response
        matchups = response.json()
        return(matchups)
    else:
        print(f"Error: {response.status_code}")

In [5]:
def get_matchups_season(league_id, season, play_off_week_start, all_rosters):
    users_df = all_rosters[all_rosters["Season"] == str(season)]
    season_matchups = pd.DataFrame()
    for i in range(18):
        week = i+1
        matchups_response = get_matchups(league_id, week_number = week)
        
        if matchups_response != []:
            matchups = {"points": [],
                        "roster_id": [],
                        "matchup_id": []}
            
            for matchup in matchups_response:
                matchups["points"].append(matchup["points"])
                matchups["roster_id"].append(matchup["roster_id"])
                matchups["matchup_id"].append(matchup["matchup_id"])

            
            matchups_df = pd.DataFrame(matchups)
            matchups_df = matchups_df.sort_values(by=['matchup_id', 'points']).reset_index(drop=True)
            matchups_df['pair'] = matchups_df.groupby('matchup_id').cumcount() + 1
            matchups_df = matchups_df.merge(users_df, on = 'roster_id')
            matchups_df = matchups_df[matchups_df["matchup_id"].notnull()] # if it's playoffs, teams that aren't included still return a matchup but shouldn't be kept

            matchups_df_wide = matchups_df.pivot(index='matchup_id', columns='pair', values=['roster_id', 'points', 'display_name', 'user_id'])
            matchups_df_wide.columns = [f"{col[0]}_{col[1]}" for col in matchups_df_wide.columns]
            matchups_df_wide["season"] = season
            matchups_df_wide["week"] = week
            matchups_df_wide["match_type"] = matchups_df_wide['week'].apply(lambda x: 'Regular Season' if x < int(play_off_week_start) else 'Playoffs')

            season_matchups = pd.concat([season_matchups, matchups_df_wide])  

    season_matchups = season_matchups[(season_matchups["points_1"] > 0) & (season_matchups["points_2"] > 0)]
    season_matchups['user_pair'] = season_matchups.apply(lambda row: tuple(sorted([row['display_name_1'], row['display_name_2']])), axis=1)
    season_matchups["Winner"] = season_matchups.apply(lambda row: row['display_name_1'] if row['points_1'] > row['points_2'] else row['display_name_2'], axis=1)
    season_matchups["Loser"] = season_matchups.apply(lambda row: row['display_name_2'] if row['points_1'] > row['points_2'] else row['display_name_1'], axis=1)

    return(season_matchups)


# Starting from scratch here is the order of operations to get the info

## 1. here we get the league info for each season:

In [6]:
league_info = get_all_league_info(1073659471932538880) # dataframe with league_id, season, playoff_week_start
league_info

,league_id,season,playoff_week_start
0,1073659471932538880,2024,15
1,936029369854734336,2023,15
2,851216576215617536,2022,15
3,650041395410722816,2021,15
4,515606348271226880,2020,14
5,456679182605217792,2019,14


## 2. here we get the rosters for each season
- this helps account for changing users
- roster_id is also not constant year by year so we need to make sure we are merging the right user for each matchup

In [7]:
all_rosters = pd.DataFrame()

for season_id, year in zip(league_info.league_id, league_info.season):
    year_roster = get_roster_id(season_id, year)

    all_rosters = pd.concat([all_rosters, year_roster])

all_rosters.head()
    

,display_name,user_id,image_link,Season,roster_id
0,BLoesch,456677174343102464,https://sleepercdn.com/uploads/1ef1eb5c2cdbac2...,2024,1
1,jloesch,456846148506742784,https://sleepercdn.com/uploads/74511e5a39bb967...,2024,2
2,Sousdini,456865634018390016,https://sleepercdn.com/uploads/d426af324158252...,2024,3
3,charifelm,456881030087831552,https://sleepercdn.com/uploads/1ce0f19708ae549...,2024,5
4,awilley19,456939547255631872,Not found.,2024,4


## 3. here we get the matchups for a season

In [8]:
season_matchups = get_matchups_season(league_info["league_id"][0], league_info["season"][0], league_info["playoff_week_start"][0], all_rosters)

season_matchups.head()

,roster_id_1,roster_id_2,points_1,points_2,display_name_1,display_name_2,user_id_1,user_id_2,season,week,match_type,user_pair,Winner,Loser
matchup_id,,,,,,,,,,,,,,
1,5,3,115.96,145.46,charifelm,Sousdini,456881030087831552,456865634018390016,2024,1,Regular Season,"(Sousdini, charifelm)",Sousdini,charifelm
2,8,10,147.76,150.76,Aiirik91,geocesar96,757636343194050560,460258882422304768,2024,1,Regular Season,"(Aiirik91, geocesar96)",geocesar96,Aiirik91
3,4,7,123.86,124.8,awilley19,TheGoldenClutch,456939547255631872,458263550834503680,2024,1,Regular Season,"(TheGoldenClutch, awilley19)",TheGoldenClutch,awilley19
4,2,9,143.36,144.98,jloesch,dompappa,456846148506742784,459487653503430656,2024,1,Regular Season,"(dompappa, jloesch)",dompappa,jloesch
5,1,6,119.3,150.1,BLoesch,APTwoOne,456677174343102464,456971412364587008,2024,1,Regular Season,"(APTwoOne, BLoesch)",APTwoOne,BLoesch


## 4. Finally get the matchups for every possible season
- one thing I should add is checking if the week started
    - this would mostly be relevant for the current season
    - we only want to include completed games

In [9]:
for index, row in league_info.iterrows():
    print(row["league_id"], row["season"], row["playoff_week_start"])

1073659471932538880 2024 15
936029369854734336 2023 15
851216576215617536 2022 15
650041395410722816 2021 15
515606348271226880 2020 14
456679182605217792 2019 14


In [10]:
all_matchups = pd.DataFrame()

for id, season, playoff_start in zip(league_info.league_id, league_info.season, league_info.playoff_week_start):
    season_matchups = get_matchups_season(id, season, playoff_start, all_rosters)

    all_matchups = pd.concat([all_matchups, season_matchups])
    

C:\Users\Brandon Loesch\AppData\Local\Temp\ipykernel_24852\3343561248.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  season_matchups = pd.concat([season_matchups, matchups_df_wide])
C:\Users\Brandon Loesch\AppData\Local\Temp\ipykernel_24852\3343561248.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  season_matchups = pd.concat([season_matchups, matchups_df_wide])
C:\Users\Brandon Loesch\AppData\Local\Temp\ipykernel_24852\3343561248.py:31: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, t

In [300]:
all_matchups.to_csv("test_matchups_new.csv")

In [308]:
all_matchups[all_matchups["user_pair"] == ("BLoesch", "jloesch")].sort_values(by = "season")

,roster_id_1,roster_id_2,points_1,points_2,display_name_1,display_name_2,user_id_1,user_id_2,season,week,...,Winner,Loser,roster_id_1.0,roster_id_2.0,points_1.0,points_2.0,display_name_1.0,display_name_2.0,user_id_1.0,user_id_2.0
matchup_id,,,,,,,,,,,,,,,,,,,,,
5.0,1,2,141.56,184.32,BLoesch,jloesch,456677174343102464,456846148506742784,2019,2,...,jloesch,BLoesch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5.0,2,1,121.1,125.08,jloesch,BLoesch,456846148506742784,456677174343102464,2019,11,...,BLoesch,jloesch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,1,2,138.66,138.78,BLoesch,jloesch,456677174343102464,456846148506742784,2020,4,...,jloesch,BLoesch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,2,1,106.6,168.46,jloesch,BLoesch,456846148506742784,456677174343102464,2020,13,...,BLoesch,jloesch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,2,1,134.66,172.0,jloesch,BLoesch,456846148506742784,456677174343102464,2021,1,...,BLoesch,jloesch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4.0,2,1,112.56,149.92,jloesch,BLoesch,456846148506742784,456677174343102464,2021,10,...,BLoesch,jloesch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2.0,2,1,169.0,183.16,jloesch,BLoesch,456846148506742784,456677174343102464,2021,16,...,BLoesch,jloesch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.0,2,1,125.1,146.46,jloesch,BLoesch,456846148506742784,456677174343102464,2022,4,...,BLoesch,jloesch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1.0,2,1,152.94,178.2,jloesch,BLoesch,456846148506742784,456677174343102464,2022,13,...,BLoesch,jloesch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#testing

In [75]:
league_id = 936029369854734336
season = 2023
play_off_week_start = 15

users_df = all_rosters[all_rosters["Season"] == str(season)]
season_matchups = pd.DataFrame()
for i in range(20):
    week = i+1
    matchups_response = get_matchups(league_id, week_number = week)

    if matchups_response != []:
        matchups = {"points": [],
                    "roster_id": [],
                    "matchup_id": []}
        
        for matchup in matchups_response:
            matchups["points"].append(matchup["points"])
            matchups["roster_id"].append(matchup["roster_id"])
            matchups["matchup_id"].append(matchup["matchup_id"])

        
        matchups_df = pd.DataFrame(matchups)
        matchups_df = matchups_df.sort_values(by=['matchup_id', 'points']).reset_index(drop=True)
        matchups_df['pair'] = matchups_df.groupby('matchup_id').cumcount() + 1
        matchups_df = matchups_df.merge(users_df, on = 'roster_id')
        matchups_df = matchups_df[matchups_df["matchup_id"].notnull()] # if it's playoffs, teams that aren't included still return a matchup but shouldn't be kept
        matchups_df['matchup_id'] = matchups_df['matchup_id'].astype(int)

        matchups_df_wide = matchups_df.pivot(index='matchup_id', columns='pair', values=['roster_id', 'points', 'display_name', 'user_id'])
        matchups_df_wide.columns = [f"{col[0]}_{str(col[1])}" for col in matchups_df_wide.columns]
        matchups_df_wide["season"] = season
        matchups_df_wide["week"] = week
        matchups_df_wide["match_type"] = matchups_df_wide['week'].apply(lambda x: 'Regular Season' if x < int(play_off_week_start) else 'Playoffs')

        season_matchups = pd.concat([season_matchups, matchups_df_wide])  

    season_matchups = season_matchups[(season_matchups["points_1"] > 0) & (season_matchups["points_2"] > 0)]
    season_matchups['user_pair'] = season_matchups.apply(lambda row: tuple(sorted([row['display_name_1'], row['display_name_2']])), axis=1)
    season_matchups["Winner"] = season_matchups.apply(lambda row: row['display_name_1'] if row['points_1'] > row['points_2'] else row['display_name_2'], axis=1)
    season_matchups["Loser"] = season_matchups.apply(lambda row: row['display_name_2'] if row['points_1'] > row['points_2'] else row['display_name_1'], axis=1)

In [76]:
season_matchups

,roster_id_1,roster_id_2,points_1,points_2,display_name_1,display_name_2,user_id_1,user_id_2,season,week,...,Winner,Loser,roster_id_1.0,roster_id_2.0,points_1.0,points_2.0,display_name_1.0,display_name_2.0,user_id_1.0,user_id_2.0
matchup_id,,,,,,,,,,,,,,,,,,,,,
1,4,3,108.46,168.7,awilley19,Sousdini,456939547255631872,456865634018390016,2023,1,...,Sousdini,awilley19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,8,134.14,150.84,charifelm,Aiirik91,456881030087831552,757636343194050560,2023,1,...,Aiirik91,charifelm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2,10,120.5,148.32,jloesch,geocesar96,456846148506742784,460258882422304768,2023,1,...,geocesar96,jloesch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,1,125.1,130.72,APTwoOne,BLoesch,456971412364587008,456677174343102464,2023,1,...,BLoesch,APTwoOne,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,9,7,105.8,143.48,dompappa,TheGoldenClutch,459487653503430656,458263550834503680,2023,1,...,TheGoldenClutch,dompappa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1,4,8,144.34,153.06,awilley19,Aiirik91,456939547255631872,757636343194050560,2023,16,...,Aiirik91,awilley19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,9,112.54,167.06,BLoesch,dompappa,456677174343102464,459487653503430656,2023,16,...,dompappa,BLoesch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10,3,139.74,211.94,geocesar96,Sousdini,460258882422304768,456865634018390016,2023,16,...,Sousdini,geocesar96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [83]:
matchups_response = get_matchups(936029369854734336, week_number = 17)

if matchups_response != []:
    matchups = {"points": [],
                "roster_id": [],
                "matchup_id": []}
    
    for matchup in matchups_response:
        matchups["points"].append(matchup["points"])
        matchups["roster_id"].append(matchup["roster_id"])
        matchups["matchup_id"].append(matchup["matchup_id"])

    
    matchups_df = pd.DataFrame(matchups)
    matchups_df = matchups_df.sort_values(by=['matchup_id', 'points']).reset_index(drop=True)
    matchups_df['pair'] = matchups_df.groupby('matchup_id').cumcount() + 1
    matchups_df = matchups_df.merge(users_df, on = 'roster_id')
    matchups_df = matchups_df[matchups_df["matchup_id"].notnull()]
    matchups_df['matchup_id'] = matchups_df['matchup_id'].astype(int)

    matchups_df_wide = matchups_df.pivot(index='matchup_id', columns='pair', values=['roster_id', 'points', 'display_name', 'user_id'])
    matchups_df_wide.columns = [f"{col[0]}_{int(col[1])}" for col in matchups_df_wide.columns]
    matchups_df_wide["season"] = season
    matchups_df_wide["week"] = week
    matchups_df_wide["match_type"] = matchups_df_wide['week'].apply(lambda x: 'Regular Season' if x < int(play_off_week_start) else 'Playoffs')

In [84]:
matchups_df

,points,roster_id,matchup_id,pair,display_name,user_id,image_link,Season
0,137.98,9,1,1.0,dompappa,459487653503430656,https://sleepercdn.com/uploads/8d4af248c58d527...,2023
1,164.28,8,1,2.0,Aiirik91,757636343194050560,https://sleepercdn.com/uploads/eae855d31520c6a...,2023
2,100.68,4,2,1.0,awilley19,456939547255631872,Not found.,2023
3,192.58,1,2,2.0,BLoesch,456677174343102464,Not found.,2023


In [85]:
matchups_df_wide 

,roster_id_1,roster_id_2,points_1,points_2,display_name_1,display_name_2,user_id_1,user_id_2,season,week,match_type
matchup_id,,,,,,,,,,,
1,9,8,137.98,164.28,dompappa,Aiirik91,459487653503430656,757636343194050560,2023,20,Playoffs
2,4,1,100.68,192.58,awilley19,BLoesch,456939547255631872,456677174343102464,2023,20,Playoffs


In [74]:
[f"{col[0]}_{str(col[1])}" for col in matchups_df_wide.columns]

['roster_id_1.0',
 'roster_id_2.0',
 'points_1.0',
 'points_2.0',
 'display_name_1.0',
 'display_name_2.0',
 'user_id_1.0',
 'user_id_2.0']

In [11]:
matchups = pd.read_csv(r"D:\Brandon Loesch\Sleeper_app\test_matchups_new.csv")

# trying to recreate matrix function but for points scored

In [12]:
matchups

,matchup_id,roster_id_1,roster_id_2,points_1,points_2,display_name_1,display_name_2,user_id_1,user_id_2,season,...,Winner,Loser,roster_id_1.0,roster_id_2.0,points_1.0,points_2.0,display_name_1.0,display_name_2.0,user_id_1.0,user_id_2.0
0,1.0,5,3,115.96,145.46,awilley19,Sousdini,456939547255631872,456865634018390016,2024,...,Sousdini,awilley19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.0,8,10,147.76,150.76,dompappa,Aiirik91,459487653503430656,757636343194050560,2024,...,Aiirik91,dompappa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.0,4,7,123.86,124.80,charifelm,TheGoldenClutch,456881030087831552,458263550834503680,2024,...,TheGoldenClutch,charifelm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4.0,2,9,143.36,144.98,jloesch,geocesar96,456846148506742784,460258882422304768,2024,...,geocesar96,jloesch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5.0,1,6,119.30,150.10,BLoesch,APTwoOne,456677174343102464,456971412364587008,2024,...,APTwoOne,BLoesch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
405,1.0,7,6,178.22,185.62,robcummings,APTwoOne,457718691669209088,456971412364587008,2019,...,APTwoOne,robcummings,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
406,2.0,4,1,161.20,162.76,charifelm,BLoesch,456881030087831552,456677174343102464,2019,...,BLoesch,charifelm,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
407,3.0,9,2,136.68,184.18,dompappa,jloesch,459487653503430656,456846148506742784,2019,...,jloesch,dompappa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
408,4.0,3,10,120.40,145.76,Sousdini,geocesar96,456865634018390016,460258882422304768,2019,...,geocesar96,Sousdini,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
head_to_head = pd.DataFrame(np.nan, 
                            index = sorted(set(all_rosters[all_rosters["Season"] == "2024"]['display_name'])),
                            columns = sorted(set(all_rosters[all_rosters["Season"] == "2024"]['display_name'])))


In [14]:
teams = sorted(set(all_rosters[all_rosters["Season"] == "2024"]['display_name']))

In [15]:
scores = {team: {opponent: 0 for opponent in teams} for team in teams}

In [16]:
scores

{'APTwoOne': {'APTwoOne': 0,
  'Aiirik91': 0,
  'BLoesch': 0,
  'Sousdini': 0,
  'TheGoldenClutch': 0,
  'awilley19': 0,
  'charifelm': 0,
  'dompappa': 0,
  'geocesar96': 0,
  'jloesch': 0},
 'Aiirik91': {'APTwoOne': 0,
  'Aiirik91': 0,
  'BLoesch': 0,
  'Sousdini': 0,
  'TheGoldenClutch': 0,
  'awilley19': 0,
  'charifelm': 0,
  'dompappa': 0,
  'geocesar96': 0,
  'jloesch': 0},
 'BLoesch': {'APTwoOne': 0,
  'Aiirik91': 0,
  'BLoesch': 0,
  'Sousdini': 0,
  'TheGoldenClutch': 0,
  'awilley19': 0,
  'charifelm': 0,
  'dompappa': 0,
  'geocesar96': 0,
  'jloesch': 0},
 'Sousdini': {'APTwoOne': 0,
  'Aiirik91': 0,
  'BLoesch': 0,
  'Sousdini': 0,
  'TheGoldenClutch': 0,
  'awilley19': 0,
  'charifelm': 0,
  'dompappa': 0,
  'geocesar96': 0,
  'jloesch': 0},
 'TheGoldenClutch': {'APTwoOne': 0,
  'Aiirik91': 0,
  'BLoesch': 0,
  'Sousdini': 0,
  'TheGoldenClutch': 0,
  'awilley19': 0,
  'charifelm': 0,
  'dompappa': 0,
  'geocesar96': 0,
  'jloesch': 0},
 'awilley19': {'APTwoOne': 0,
  'A

In [17]:
for _, row in matchups.iterrows():
    team1, team2 = row['display_name_1'], row['display_name_2']
    team1_score, team2_score = row['points_1'], row['points_2']

    if team1 not in scores.keys() or team2 not in scores.keys():
        continue

    scores[team1][team2] += team1_score
    scores[team2][team1] += team2_score

In [18]:
scores

{'APTwoOne': {'APTwoOne': 0,
  'Aiirik91': 409.2,
  'BLoesch': 1144.96,
  'Sousdini': 1497.3,
  'TheGoldenClutch': 1369.3000000000002,
  'awilley19': 976.7400000000001,
  'charifelm': 1433.82,
  'dompappa': 1129.52,
  'geocesar96': 1662.1399999999999,
  'jloesch': 1187.86},
 'Aiirik91': {'APTwoOne': 413.9,
  'Aiirik91': 0,
  'BLoesch': 628.96,
  'Sousdini': 975.0,
  'TheGoldenClutch': 655.48,
  'awilley19': 823.66,
  'charifelm': 689.7,
  'dompappa': 812.74,
  'geocesar96': 447.53999999999996,
  'jloesch': 617.9},
 'BLoesch': {'APTwoOne': 1013.74,
  'Aiirik91': 592.26,
  'BLoesch': 0,
  'Sousdini': 1318.42,
  'TheGoldenClutch': 1823.6,
  'awilley19': 1158.74,
  'charifelm': 1225.4,
  'dompappa': 1131.54,
  'geocesar96': 1287.8999999999999,
  'jloesch': 1873.48},
 'Sousdini': {'APTwoOne': 1466.0599999999997,
  'Aiirik91': 764.78,
  'BLoesch': 1286.7599999999998,
  'Sousdini': 0,
  'TheGoldenClutch': 1029.06,
  'awilley19': 1325.74,
  'charifelm': 1440.14,
  'dompappa': 1175.500000000000

In [19]:
head_to_head_scores = pd.DataFrame(
    {team: {opponent: score if score > 0 else "" 
            for opponent, score in opponents.items()}
     for team, opponents in scores.items()}
)

In [20]:
head_to_head_scores

,APTwoOne,Aiirik91,BLoesch,Sousdini,TheGoldenClutch,awilley19,charifelm,dompappa,geocesar96,jloesch
APTwoOne,,413.9,1013.74,1466.06,1388.18,853.32,1375.4,1242.76,1741.48,1164.4
Aiirik91,409.2,,592.26,764.78,503.68,653.44,622.64,810.86,457.76,625.12
BLoesch,1144.96,628.96,,1286.76,1643.82,1154.82,1089.14,961.66,1352.38,1685.44
Sousdini,1497.3,975.0,1318.42,,1016.78,1269.06,1464.28,1385.38,1251.12,1294.48
TheGoldenClutch,1369.3,655.48,1823.6,1029.06,,1390.9,965.94,877.44,1296.42,1592.56
awilley19,976.74,823.66,1158.74,1325.74,1512.1,,1480.54,1727.08,959.52,1463.7
charifelm,1433.82,689.7,1225.4,1440.14,910.16,1403.7,,1482.84,1411.78,901.82
dompappa,1129.52,812.74,1131.54,1175.5,1027.58,1554.64,1422.46,,1537.24,1395.42
geocesar96,1662.14,447.54,1287.9,1046.96,1196.64,842.66,1432.48,1450.68,,1520.54
jloesch,1187.86,617.9,1873.48,1215.56,1481.26,1424.86,772.1,1385.0,1421.3,


In [30]:
for _, row in matchups.iterrows():
    team1, team2 = row['display_name_1'], row['display_name_2']
    team1_score, team2_score = row['points_1'], row['points_2']

    if team1 not in records.keys() or team2 not in records.keys():
        continue
    
    if team1_score > team2_score:
        records[team1][team2][0] += 1  # team1 wins
        records[team2][team1][1] += 1  # team2 loses
    else:
        records[team1][team2][1] += 1  # team1 loses
        records[team2][team1][0] += 1  # team2 wins

In [31]:
records

{'APTwoOne': {'APTwoOne': [0, 0],
  'Aiirik91': [2, 4],
  'BLoesch': [8, 5],
  'Sousdini': [6, 9],
  'TheGoldenClutch': [4, 12],
  'awilley19': [8, 3],
  'charifelm': [6, 6],
  'dompappa': [4, 8],
  'geocesar96': [6, 10],
  'jloesch': [6, 7]},
 'Aiirik91': {'APTwoOne': [4, 2],
  'Aiirik91': [0, 0],
  'BLoesch': [6, 2],
  'Sousdini': [10, 2],
  'TheGoldenClutch': [6, 2],
  'awilley19': [8, 2],
  'charifelm': [4, 6],
  'dompappa': [6, 4],
  'geocesar96': [4, 2],
  'jloesch': [2, 6]},
 'BLoesch': {'APTwoOne': [5, 8],
  'Aiirik91': [2, 6],
  'BLoesch': [0, 0],
  'Sousdini': [5, 8],
  'TheGoldenClutch': [10, 7],
  'awilley19': [5, 6],
  'charifelm': [10, 2],
  'dompappa': [6, 4],
  'geocesar96': [8, 7],
  'jloesch': [13, 4]},
 'Sousdini': {'APTwoOne': [9, 6],
  'Aiirik91': [2, 10],
  'BLoesch': [8, 5],
  'Sousdini': [0, 0],
  'TheGoldenClutch': [8, 3],
  'awilley19': [9, 5],
  'charifelm': [7, 8],
  'dompappa': [3, 11],
  'geocesar96': [1, 10],
  'jloesch': [6, 6]},
 'TheGoldenClutch': {'AP

In [32]:
head_to_head_df = pd.DataFrame(
    {team: {opponent: f"{wins}-{losses}" if wins or losses else "" 
            for opponent, (wins, losses) in opponents.items()}
     for team, opponents in records.items()}
)

In [33]:
head_to_head_df

,APTwoOne,Aiirik91,BLoesch,Sousdini,TheGoldenClutch,awilley19,charifelm,dompappa,geocesar96,jloesch
APTwoOne,,4-2,5-8,9-6,12-4,3-8,6-6,8-4,10-6,7-6
Aiirik91,2-4,,2-6,2-10,2-6,2-8,6-4,4-6,2-4,6-2
BLoesch,8-5,6-2,,8-5,7-10,6-5,2-10,4-6,7-8,4-13
Sousdini,6-9,10-2,5-8,,3-8,5-9,8-7,11-3,10-1,6-6
TheGoldenClutch,4-12,6-2,10-7,8-3,,8-6,4-6,5-5,8-6,9-8
awilley19,8-3,8-2,5-6,9-5,6-8,,14-3,14-2,9-1,8-6
charifelm,6-6,4-6,10-2,7-8,6-4,3-14,,10-6,9-5,7-2
dompappa,4-8,6-4,6-4,3-11,5-5,2-14,6-10,,6-9,6-7
geocesar96,6-10,4-2,8-7,1-10,6-8,1-9,5-9,9-6,,8-7
jloesch,6-7,2-6,13-4,6-6,8-9,6-8,2-7,7-6,7-8,


In [34]:
def generate_matrix(matchups, rosters):
    teams = sorted(set(all_rosters[all_rosters["Season"] == "2024"]['display_name']))
    records = {team: {opponent: [0, 0] for opponent in teams} for team in teams}

    for _, row in matchups.iterrows():
        team1, team2 = row['display_name_1'], row['display_name_2']
        team1_score, team2_score = row['points_1'], row['points_2']

        if team1 not in records.keys() or team2 not in records.keys():
            continue
        
        if team1_score > team2_score:
            records[team1][team2][0] += 1  # team1 wins
            records[team2][team1][1] += 1  # team2 loses
        else:
            records[team1][team2][1] += 1  # team1 loses
            records[team2][team1][0] += 1  # team2 wins

    head_to_head_df = pd.DataFrame(
    {team: {opponent: f"{wins}-{losses}" if wins or losses else "" 
            for opponent, (wins, losses) in opponents.items()}
     for team, opponents in records.items()})
    
    return(head_to_head_df)